RuCodeBank, как и любой другой банк, иногда проводит маркетинговые кампании по телефону, предлагая клиентам новые услуги. Однако часы работы менеджеров, которые обзванивают клиентов, стоят дорого, а процент людей, которые в итоге соглашаются купить услугу, довольно низок. Чтобы оптимизировать процесс рекламной кампании, вам предложили сделать модель машинного обучения, которая по информации о клиенте банка предсказывала бы, заинтересуется ли он предложенной услугой.

Есть обучающая выборка train.csv с информацией о клиентах банка, которым по телефону была предложена услуга открытия депозита в банке RuCodeBank. Целевая переменная 'контакт_исход' показывает, согласился клиент на предложенную услугу или нет. Вам нужно с помощью машинного обучения для всех людей из тестовой выборки test.csv предсказать, купит клиент услугу или нет.

Файл с ответами должен содержать одну колонку 'Out' с предсказанными ответами для людей из test.csv. Ответы в виде 0 и 1. Предсказания должны идти в том же порядке, что и люди в test.csv. Пример файла с ответами находится в submission.csv.

Описания колонок из train и test частей:

– "возраст" : возраст клиента

– "профессия" : вид деятельности клиента

– "семейное положение" : семейное положение клиента

– "образование" : уровень образования клиента

– "кредит_дефолт" : имеет ли клиент невыплаченный кредит

– "жилищный_кредит" : имеет ли клиент кредит на жилье или недвижимость в залоге

– "кредит" : имеет ли клиент другие виды кредитов

– "взаимодействие" : по какому типу связи происходило взаимодействие клиента и банка

– "контакт_месяц" : месяц, в котором в последний раз было взаимодействие клиента и банка в рамках текущей маркетинговой кампании

– "контакт_день_недели" : день недели, в котором в последний раз было взаимодействие клиента и банка в рамках текущей маркетинговой кампании

– "контакт_количество" : количество взаимодействий между клиентом и банком за период текущей рекламной кампании

– "контакт_период" : количество дней, прошедших с момента взаимодействия с клиентом в рамках предыдущих рекламных кампаний (999 означает, что с клиентом у банка еще не было взаимодействий)

– "контакт_прошлый_количество" : общее количество контактов между банком и клиентом до текущей рекламной кампании

– "контакт_прошлый_исход" : купил ли клиент услугу в рамках прошлой рекламной кампании

– "контакт_исход" (только для train.csv) : целевая переменная: купил ли клиент услугу в рамках текущей рекламной кампании

Данные: https://drive.google.com/drive/folders/1knZCSJ_XpSSQjN-5R2tBUuwRV6Br9xKS?usp=sharing

Качество предсказаний оценивается по метрике F1score: https://en.wikipedia.org/wiki/F1_score.

Ваши баллы будут равны 10⋅ F1Score округленные вверх.

Из-за технической особенности Яндекс.Контеста (Невозможность иметь в одном соревновании задачи с частичными баллами и обычные задачи) баллы за посылку не начисляются в автоматическом режиме, но их можно посмотреть, нажав на отчет по этой посылке. Баллы за посылку будут вноситься в ручном режиме с задержкой пару часов.

Внимание! Баллы за посылки не окончательные, они будут перераспределены после окончания контекста на основе результатов участников.

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
path_test = 'marketing_campaign_test.csv'
path_train =  'marketing_campaign_train.csv'

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import scale
from sklearn.metrics import f1_score
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.tree import export_graphviz
from IPython.display import SVG
from graphviz import Source
from IPython.display import display, Image
from ipywidgets import interactive, IntSlider, FloatSlider, interact
import ipywidgets
from subprocess import call
import matplotlib.image as mpimg


In [ ]:
train_df = pd.read_csv(path_train)
test_df = pd.read_csv(path_test)
#train_df.fillna(value=-1, inplace=True)
train_df = train_df.iloc[:,1:-1]
cleanup_nums = {"профессия": {"администриратор": 1, "рабочий": 2, "техник/механик":3, "сфера услуг":4, "менеджмент":5, 
                                  "пенсионер":6, "предприниматель":7, "самозанятый":8, "домохозяйка":9, "безработный":10, "учащийся":11},
                "семейное_положение": {"женат/замужем": 1, "не женат/не замужем": 2, "разведен": 3},
                "образование": {"неоконченное среднее":1,"высшее":2,"среднее":3,"специальное":4,"без образования":5},
                "взаимодействие": {"мобильный":0,"домашний":1},
                "контакт_день_недели": {"пн":1,"вт":2,"ср":3,"чт":4,"пт":5}
                }
train_df.replace(cleanup_nums, inplace=True)

#train_df['контакт_период'] = scale(train_df['контакт_период'])
train_df.dropna(inplace=True)
train_df.drop_duplicates(subset = ['контакт_период','кредит_дефолт','семейное_положение','образование','кредит', 'контакт_исход','жилищный_кредит','контакт_день_недели','контакт_месяц','взаимодействие','контакт_прошлый_количество', 'контакт_прошлый_исход'], inplace=True)
train_df


,возраст,профессия,семейное_положение,образование,кредит_дефолт,жилищный_кредит,кредит,взаимодействие,контакт_месяц,контакт_день_недели,контакт_количество,контакт_период,контакт_прошлый_количество,контакт_прошлый_исход,контакт_исход
0,33,1.0,2.0,4.0,0.0,0.0,0.0,0,5,1,1,999,1,0,1
1,33,3.0,2.0,4.0,0.0,1.0,0.0,0,8,3,1,999,0,-,0
2,30,1.0,2.0,2.0,0.0,1.0,0.0,0,5,1,1,999,0,-,0
3,31,1.0,1.0,2.0,0.0,1.0,0.0,1,5,5,2,999,0,-,0
4,31,4.0,2.0,3.0,0.0,1.0,0.0,0,5,4,1,999,0,-,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31123,42,3.0,1.0,2.0,0.0,0.0,0.0,0,8,5,3,6,2,1,1
31136,50,1.0,3.0,2.0,0.0,1.0,0.0,0,11,5,3,1,2,1,0
31137,32,1.0,1.0,4.0,0.0,0.0,1.0,0,10,4,1,6,1,1,1
31154,35,3.0,1.0,4.0,0.0,1.0,1.0,0,10,3,1,9,2,0,1


In [ ]:
train_df['контакт_исход'].value_counts()

0    3590
1    2020
Name: контакт_исход, dtype: int64

In [ ]:
train_df.corr('pearson')

,возраст,профессия,семейное_положение,образование,кредит_дефолт,жилищный_кредит,кредит,взаимодействие,контакт_месяц,контакт_день_недели,контакт_количество,контакт_период,контакт_прошлый_количество,контакт_исход
возраст,1.000000,0.089906,-0.062834,-0.126504,-0.001140,0.001577,-0.033797,-0.026356,0.088012,-0.021520,-0.012104,-0.056131,0.045471,0.042774
профессия,0.089906,1.000000,0.001970,-0.042858,0.018209,-0.003891,-0.043736,-0.031566,0.067652,-0.000873,-0.040232,-0.059798,0.063095,0.054308
семейное_положение,-0.062834,0.001970,1.000000,0.019289,-0.018096,0.003360,-0.010232,0.007476,-0.044594,0.006303,0.034039,0.084514,-0.089962,-0.085212
образование,-0.126504,-0.042858,0.019289,1.000000,0.020276,0.004752,-0.007678,-0.012244,0.011666,0.014393,0.015316,0.042277,-0.038105,-0.028219
кредит_дефолт,-0.001140,0.018209,-0.018096,0.020276,1.000000,-0.020524,-0.011502,-0.011243,0.018561,0.013681,-0.006714,0.008731,-0.001657,-0.014166
жилищный_кредит,0.001577,-0.003891,0.003360,0.004752,-0.020524,1.000000,0.032977,0.002466,0.011986,-0.004526,-0.006178,-0.008521,0.021897,-0.002904
кредит,-0.033797,-0.043736,-0.010232,-0.007678,-0.011502,0.032977,1.000000,0.016227,-0.064845,-0.001265,0.056985,0.130957,-0.147774,-0.122528
взаимодействие,-0.026356,-0.031566,0.007476,-0.012244,-0.011243,0.002466,0.016227,1.000000,-0.037674,-0.004470,0.153309,0.191127,-0.241047,-0.150030
контакт_месяц,0.088012,0.067652,-0.044594,0.011666,0.018561,0.011986,-0.064845,-0.037674,1.000000,0.012169,-0.050951,-0.082418,0.073660,0.017191
контакт_день_недели,-0.021520,-0.000873,0.006303,0.014393,0.013681,-0.004526,-0.001265,-0.004470,0.012169,1.000000,-0.020866,0.005296,-0.011795,0.020700


In [ ]:
def preprocessing(df):
  df = df.iloc[:,1:]
  clean = {"профессия": {"администриратор": 1, "рабочий": 2, "техник/механик":3, "сфера услуг":4, "менеджмент":5, 
                                  "пенсионер":6, "предприниматель":7, "самозанятый":8, "домохозяйка":9, "безработный":10, "учащийся":11},
                "семейное_положение": {"женат/замужем": 1, "не женат/не замужем": 2, "разведен": 3},
                "образование": {"неоконченное среднее":1,"высшее":2,"среднее":3,"специальное":4,"без образования":5},
                "взаимодействие": {"мобильный":0,"домашний":1},
                "контакт_день_недели": {"пн":1,"вт":2,"ср":3,"чт":4,"пт":5},
                "контакт_прошлый_исход": {"0":0,"1":1,"-":-1}
                }
  df.replace(clean, inplace=True)
  df.fillna(value=-1, inplace=True)
  df['контакт_период'] = scale(df['контакт_период'])
  return df

In [ ]:
train_y = train_df['контакт_исход']
train_x = train_df.iloc[:,:-3]
X_train, X_test, y_train, y_test = train_test_split(train_x, train_y, test_size=0.20, random_state=42)

In [ ]:
dtree = DecisionTreeClassifier(random_state = 0,max_depth=17, min_samples_split=113, min_samples_leaf=1)
dtree.fit(X_train, y_train)
print(classification_report(y_test, dtree.predict(X_test)))

              precision    recall  f1-score   support

           0       0.72      0.80      0.76       722
           1       0.55      0.44      0.49       400

    accuracy                           0.67      1122
   macro avg       0.63      0.62      0.62      1122
weighted avg       0.66      0.67      0.66      1122



In [ ]:
train_df = pd.read_csv(path_train)
test_df = pd.read_csv(path_test)
#train_df.fillna(value=-1, inplace=True)
train_df = train_df.iloc[:,1:-1]
cleanup_nums = {"профессия": {"администриратор": 1, "рабочий": 2, "техник/механик":3, "сфера услуг":4, "менеджмент":5, 
                                  "пенсионер":6, "предприниматель":7, "самозанятый":8, "домохозяйка":9, "безработный":10, "учащийся":11},
                "семейное_положение": {"женат/замужем": 1, "не женат/не замужем": 2, "разведен": 3},
                "образование": {"неоконченное среднее":1,"высшее":2,"среднее":3,"специальное":4,"без образования":5},
                "взаимодействие": {"мобильный":0,"домашний":1},
                "контакт_день_недели": {"пн":1,"вт":2,"ср":3,"чт":4,"пт":5}
                }
train_df.replace(cleanup_nums, inplace=True)

#train_df['контакт_период'] = scale(train_df['контакт_период'])
#train_df.dropna(inplace=True)
#train_df.drop_duplicates(subset = ['контакт_период','кредит_дефолт','семейное_положение','образование','кредит', 'контакт_исход','жилищный_кредит','контакт_день_недели','контакт_месяц','взаимодействие','контакт_прошлый_количество', 'контакт_прошлый_исход'], inplace=True)
train_df
X_train, X_test, y_train, y_test = train_test_split(train_x, train_y, test_size=0.20, random_state=42)
print(classification_report(y_test, dtree.predict(X_test)))

              precision    recall  f1-score   support

           0       0.72      0.80      0.76       722
           1       0.55      0.44      0.49       400

    accuracy                           0.67      1122
   macro avg       0.63      0.62      0.62      1122
weighted avg       0.66      0.67      0.66      1122



In [ ]:
max_depth=np.arange(10,20)
min_samples_leaf=np.arange(2,10)
min_samples_split=np.arange(2,10)
crit=["gini", "entropy"]
split=["best", "random"]
#Convert to dictionary
hyperparameters = dict(max_depth=max_depth, min_samples_leaf=min_samples_leaf, min_samples_split=min_samples_split,criterion=crit,splitter=split)
tree_1 = DecisionTreeClassifier()
#Use GridSearch
clf = GridSearchCV(tree, hyperparameters, cv=5, scoring='f1')
#Fit the model
best_model = clf.fit(X_train,y_train)
#Print The value of best Hyperparameters
print('Best leaf_size:', best_model.best_estimator_.get_params()['leaf_size'])
print('Best p:', best_model.best_estimator_.get_params()['p'])
print('Best n_neighbors:', best_model.best_estimator_.get_params()['n_neighbors'])

In [ ]:

@interact
def plot_tree(crit=["gini", "entropy"],
              split=["best", "random"],
              depth=IntSlider(min=1, max=300, value=2, continuous_update=False),
              min_split=IntSlider(min=2, max=300, value=2, continuous_update=False),
              min_leaf=IntSlider(min=1, max=100, value=1, continuous_update=False)):
  estimator = DecisionTreeClassifier(random_state=0,
                                     criterion=crit,
                                     splitter=split,
                                     max_depth=depth,
                                     min_samples_leaf=min_leaf,
                                     min_samples_split=min_split
                                     )
  estimator.fit(X_train, y_train)
  print('Decision Tree Training F1: {:.3f}'.format(f1_score(y_train, estimator.predict(X_train))))
  print('Decision Tree Testing F1: {:.3f}'.format(f1_score(y_test, estimator.predict(X_test))))
  graph = Source(tree.export_graphviz(estimator,out_file=None,
                                      feature_names = X_train.columns,
                                      class_names=['stay','left'],
                                      filled=True))
  #display(Image(data=graph.pipe(format='png')))
  return estimator

interactive(children=(Dropdown(description='crit', options=('gini', 'entropy'), value='gini'), Dropdown(descri…

In [ ]:
@interact
def plot_tree_rf(crit=["gini", "entropy"],
              bootstrap=["True", "False"],
              depth=IntSlider(min=1, max=300, value=3, continuous_update=False),
              min_split=IntSlider(min=2, max=300, value=2, continuous_update=False),
              min_leaf=IntSlider(min=1, max=5, value=1, continuous_update=False),
              forests=IntSlider(min=1, max=400, value=100, continuous_update=False)):
  estimator = RandomForestClassifier(random_state=1,
                                     criterion=crit,
                                     bootstrap=bootstrap,
                                     n_estimators=forests,
                                     max_depth=depth,
                                     min_samples_leaf=min_leaf,
                                     min_samples_split=min_split,
                                     n_jobs = -1,
                                     verbose=False
                                     )
  estimator.fit(X_train, y_train)
  print('Random Forest Training f1: {:.3f}'.format(f1_score(y_train, estimator.predict(X_train))))
  print('Random Forest Testing f1: {:.3f}'.format(f1_score(y_test, estimator.predict(X_test))))
  num_tree = estimator.estimators_[0]
  graph = Source(tree.export_graphviz(num_tree,out_file=None,
                                      feature_names = X_train.columns,
                                      class_names=['stay','left'],
                                      filled=True))
  #display(Image(data=graph.pipe(format='png')))
  return estimator

In [ ]:
rfc = RandomForestClassifier(bootstrap='True', ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=37, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=45, n_jobs=-1,
                       oob_score=False, random_state=1, verbose=False,
                       warm_start=False)
rfc.fit(X_train, y_train)
print(classification_report(y_test, rfc.predict(X_test)))

              precision    recall  f1-score   support

           0       0.91      0.97      0.94      5529
           1       0.55      0.27      0.37       709

    accuracy                           0.89      6238
   macro avg       0.73      0.62      0.65      6238
weighted avg       0.87      0.89      0.88      6238



In [ ]:
mlp = MLPClassifier(activation='relu',
              hidden_layer_sizes=(155,),
              learning_rate_init=0.001, max_iter=50,random_state=1, solver='adam')
mlp.fit(X_train, y_train)
print(classification_report(y_test, mlp.predict(X_test)))

              precision    recall  f1-score   support

           0       0.78      0.35      0.48       722
           1       0.41      0.83      0.55       400

    accuracy                           0.52      1122
   macro avg       0.60      0.59      0.52      1122
weighted avg       0.65      0.52      0.51      1122

